## Give Me Some Credit !

##### Dans cette partie on charge nos fichiers csv en faisant attention à bien mettre nos fichiers de kaggle cs-train et cs-test dans le dossier handson-ml !

In [66]:
from bigml.api import BigML
import numpy as np
from pandas import DataFrame, read_csv


fulltrain=read_csv('./cs-training.csv')
test=read_csv('./cs-test.csv')
print("List of all the column names:\n")
for col in fulltrain.columns:
    print(col)

List of all the column names:

Unnamed: 0
SeriousDlqin2yrs
RevolvingUtilizationOfUnsecuredLines
age
NumberOfTime30-59DaysPastDueNotWorse
DebtRatio
MonthlyIncome
NumberOfOpenCreditLinesAndLoans
NumberOfTimes90DaysLate
NumberRealEstateLoansOrLines
NumberOfTime60-89DaysPastDueNotWorse
NumberOfDependents


##### Grâce à nos manipulations, nous allons pouvoir opérer sur le fichier fulltrain et le fichier test en même temps pour ainsi faire en sorte qu'ils aient les mêmes features.

In [67]:
#On renomme la première colonne actuellement sans nom, en 'Id'

for df in [fulltrain, test]:
    df.columns.values[0]='Id'

##### On créé une nouvelle colonne 'MonthlyDebt', avec la fonction '.insert' on va pouvoir insérer cette nouvelle colonne en 7e position par exemple (0 étant la 1re position) juste après 'DebtRatio' et mettre comme valeurs par défaut '0' sur chaque ligne.

In [68]:
#Méthode insert(loc, column, value, allow_duplicates=False)

for df in [fulltrain, test]:
    df.insert(6,"MonthlyDebt",0)
    
#On affiche les 20 premières lignes afin de vérifier la création de notre nouvelle colonne dans notre tableau
    
fulltrain.head(20)

,Id,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyDebt,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,0,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,0,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,0,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,0,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,0,63588.0,7,0,1,0,0.0
5,6,0,0.213179,74,0,0.375607,0,3500.0,3,0,1,0,1.0
6,7,0,0.305682,57,0,5710.000000,0,NaN,8,0,3,0,0.0
7,8,0,0.754464,39,0,0.209940,0,3500.0,8,0,0,0,0.0
8,9,0,0.116951,27,0,46.000000,0,NaN,2,0,0,0,NaN
9,10,0,0.189169,57,0,0.606291,0,23684.0,9,0,4,0,2.0


##### On définit la fonction MonthlyDebtCalc que l'ont pourra appliquer sur notre colonne correspondante. On regarde DebtRatio, si on a un résultat dans cette colonne qui est supérieur à 100, on remplace la valeur correspondante dans la colonne MonthlyDebt par cette valeur. Sinon, on multiplie DebtRatio par MonthlyIncome puis on remplace la valeur obtenue toujours dans notre colonne MonthlyDebt.

In [69]:
#Méthode apply(func, axis=0, broadcast=None, raw=False, reduce=None, result_type=None, args=(), **kwds)

def MonthlyDebtCalc(row):
    if row['DebtRatio'] > 100:
        row['MonthlyDebt']=row['DebtRatio']
    else:
        row['MonthlyDebt']=row['DebtRatio']*row['MonthlyIncome']
    return row

In [70]:
#On utilise notre fonction MonthlyDebtCalc avec apply

for df in [fulltrain, test]:
    df['MonthlyDebt']=df[['MonthlyDebt','DebtRatio','MonthlyIncome']].apply(MonthlyDebtCalc, axis=1)

In [71]:
#On regarde les 20 premières lignes de la colonne demandée

fulltrain['MonthlyDebt'].head(20)

0      7323.197016
1       316.878123
2       258.914887
3       118.963951
4      1584.975094
5      1314.624392
6      5710.000000
7       734.790059
8              NaN
9     14359.393699
10      773.690525
11     3455.468469
12     3715.701650
13     5246.617034
14      477.000000
15     2384.790109
16     2058.000000
17     1656.811728
18     1731.472112
19       21.934132
Name: MonthlyDebt, dtype: float64

##### Création d'une colonne 'IsOld' par raport à 'age'. Celle-ci définira dans notre colonne si la personne correspondante est âgée de plus de 70 ans ou non.

In [72]:
#Nouvelle colonne IsOld par rapport à 'age'

for df in [fulltrain, test]:
    df.insert(4,"IsOld",0)

##### On définit notre fonction AgeCalc. Si la valeur dans 'age' est strictement supérieure à 70, la valeur dans 'IsOld' sera 1, dans si elle est inférieure à 70, cette valeur sera 0 dans la colonne 'IsOld'.

In [73]:
#Fonction AgeCalc qui définit si la personne est âgé de plus de 70ans ou non

def AgeCalc(row):
    if row['age'] > 70:
        row['IsOld']=1
    else:
        row['IsOld']=0
    return row

In [74]:
#On utilise notre fonction avec la méthode apply sur IsOld

for df in [fulltrain, test]:
    df['IsOld']=df[['IsOld','age']].apply(AgeCalc,axis=1)

##### Dans nos colonnes 'NumberOfTime...' on retrouve plusieurs valeurs aberrantes (96,98), on va donc les remplacer par NaN.

In [75]:
#Changer les valeurs aberrantes '96' et '98' des colonnes NumberOfTime par NaN

for df in [fulltrain, test]:
    df['NumberOfTime30-59DaysPastDueNotWorse']=df['NumberOfTime30-59DaysPastDueNotWorse'].replace(96,np.nan)
    df['NumberOfTime30-59DaysPastDueNotWorse']=df['NumberOfTime30-59DaysPastDueNotWorse'].replace(98,np.nan)
    df['NumberOfTime60-89DaysPastDueNotWorse']=df['NumberOfTime60-89DaysPastDueNotWorse'].replace(96,np.nan)
    df['NumberOfTime60-89DaysPastDueNotWorse']=df['NumberOfTime60-89DaysPastDueNotWorse'].replace(98,np.nan)
    df['NumberOfTimes90DaysLate']=df['NumberOfTimes90DaysLate'].replace(96,np.nan)
    df['NumberOfTimes90DaysLate']=df['NumberOfTimes90DaysLate'].replace(98,np.nan)

##### On créé la colonne MonthlyPerPerson.

In [76]:
#Création colonne 'MonthlyPerPerson'

for df in [fulltrain, test]:
    df.insert(10,"MonthlyPerPerson",0)
    df['MonthlyPerPerson'] = df['MonthlyIncome'] / ( df['NumberOfDependents'] + 1 )

##### On créé la colonne 'MonthlyBalance' qui est la différence entre 'MonthlyIncome' et 'MonthlyPerPerson'.

In [77]:
#Création de la colonne 'MonthlyBalance'

for df in [fulltrain, test]:
    df.insert(11,"MonthlyBalance",0)
    df['MonthlyBalance'] = df['MonthlyIncome'] - df['MonthlyPerPerson']

##### On souhaite changer les valeurs NaN de notre colonne 'NumberOfDependents' par des 0, on utilise la fonction '.fillna(0)'.

In [78]:
#Remplacer les valeurs NA de la colonne NumberOfDependents par 0

for df in [fulltrain, test]:
    df['NumberOfDependents'].fillna(0)

##### On créé une nouvelle colonne nommée 'NumberTimePond' qui pondère chaque valeurs des colonnes 'NumberOfTime...' par 3, 2, 1 respectivement 90Days, 60-89Days, 30-59Days.

In [79]:
#Nouvelle colonne NumberTimePond 

for df in [fulltrain, test]:
    df.insert(5,"NumberTimePond",0)
    df['NumberTimePond'] = 3*df['NumberOfTimes90DaysLate'] + 2*df['NumberOfTime60-89DaysPastDueNotWorse']+ df['NumberOfTime30-59DaysPastDueNotWorse']

##### On vérifie que l'on obtient pas de valeurs maximales aberrantes en procédant comme suit, avec la colonne souhaitée.

```python
print(df['NumberOfTimes90DaysLate'].describe())```

##### Une fois tout cela fait, on peut enregistrer nos modifications de fulltrain et test, grâce à '.to_csv(' ')', en '.csv', on les appellera respesctivement 'GMSCfulltrainmodif.csv' et 'GMSCtestmodif.csv' par exemple.

In [85]:
#On enregistre nos modifications de fulltrain et test

fulltrain.to_csv('GMSCtrainmodif_v2.csv')
test.to_csv('GMSCtestmodif_v2.csv')

##### On peut ainsi charger notre fulltrainmodif dans notre source BigML puis effectuer des prédictions.
##### Après l'avoir mis dans notre source BigML, on créé notre dataset sur notre fulltrainmodif sur lequel on pourra créer un modèle Ensemble puis le spliter en 80-20 par exemple. 

In [91]:
#On met l'ID BigML, la key et l'id du projet

api = BigML('CorantinO', '77225993ed8676123a26c4cdaa13cfadadf71237', project='project/5d94a41242129f2e16000244')

#On importe nos deux fichiers csv fulltrainmodif et testmodif dans la source BigML

source1 = api.create_source('GMSCtrainmodif_v2.csv')
source2 = api.create_source('GMSCtestmodif_v2.csv')

HTTP request error: ('Connection aborted.', OSError("(32, 'EPIPE')"))


2019-10-25 09:07:46,090: HTTP request error: ('Connection aborted.', OSError("(32, 'EPIPE')"))


HTTP request error: ('Connection aborted.', OSError("(32, 'EPIPE')"))


2019-10-25 09:10:45,928: HTTP request error: ('Connection aborted.', OSError("(32, 'EPIPE')"))


In [ ]:
#On créé notre dataset fulltrainmodif et testmodif

origin_dataset1 = api.create_dataset(source1)
origin_dataset2 = api.create_dataset(source2)
ensemble = api.create_ensemble(origin_dataset1)

##### Grâce à ces lignes de code, on va pouvoir effectuer un split de notre dataset, ici en 80-20.
```python
train_dataset = api.create_dataset(
    origin_dataset, {"name": " Xset | Training",
                     "sample_rate": 0.8, "seed": "my seed"})
test_dataset = api.create_dataset(
    origin_dataset, {"name": " Xset | Test",
                     "sample_rate": 0.8, "seed": "my seed",
                     "out_of_bag": True})
```
##### "out_of_bag" va nous permettre de prendre les 20% restant pour notre split sur le test, on laisse donc '0.8' pour "sample_rate" aux deux.

In [62]:
#On split notre dataset fulltrainmodif en 80-20

fulltrainm_dataset = api.create_dataset(
    origin_dataset1, {"name": "GMSCfulltrainm_v2 | Training", "sample_rate": 0.8, "seed": "my seed"})
test_dataset = api.create_dataset(
    origin_dataset1, {"name": "GMSCfulltrainm_v2 | Test", "sample_rate": 0.8, "seed": "my seed", "out_of_bag": True})

##### Lors de la création du modèle Ensemble, on n'oublie pas de préciser l' "objective_field" qui sera la donnée sur laquelle on fait une prédiction.
##### Puis on fait un batch prediction sur nos 20% pour évaluer notre modèle.

In [63]:
#Création d'un modèle Ensemble sur la partie Training du dataset 

ensemble = api.create_ensemble(fulltrainm_dataset , {"objective_field" : "SeriousDlqin2yrs"})


In [64]:
#Création batch prediction sur la partie Test du dataset

batch_prediction3 = api.create_batch_prediction(ensemble, test_dataset, {"all_fields": True, "Probabilities": True})
api.ok(batch_prediction3)

True

##### Si on le souhaite, on peut télécharger notre batch prediction dans le dossier handson-ml avec la manipulation suivante:

In [65]:

#Téléchargement du batch prediction

api.download_batch_prediction(batch_prediction3, filename='BatchPrediction/GMSCfulltrainm_Prediction_Ensemble_v2.csv')


'BatchPrediction/GMSCfulltrainm_Prediction_Ensemble_v2.csv'

##### On peut créer une prédiction selon un modèle et des critères et les obtenir ici :
```Python
prediction = api.create_prediction(ensemble,{"DebtRatio": 20 , "age": 20})
api.pprint(prediction)
```
##### Par la suite il nous reste juste à appliquer un batch prediction sur notre testfull(testmodif) qui contient les mêmes features que sur le train et l'envoyer sur Kaggle. Ici on a mis 'origin_dataset2' qui correspond à notre testmodif et on enregistre notre prediction dans notre dossier handson-ml classé dans batch_prediction.

In [24]:
#On peut remplacer " all_fields " par " 'output_fields':['Id'] " pour n'obtenir que les colonnes 'Id' et celles des probabilités
batch_prediction2 = api.create_batch_prediction(ensemble, origin_dataset2, {"output_fields": ['Id'], "probability": True})
api.ok(batch_prediction2)

True

##### On a la colonne 'SeriousDlqin2yrs' qui est toujours présente ce qui nous pose problème... On va la supprimer de notre batch_prediction

In [25]:
api.download_batch_prediction(batch_prediction2, filename='BatchPrediction/GMSCfulltrainm_Prediction2_Ensemble.csv')

'BatchPrediction/GMSCfulltrainm_Prediction2_Ensemble.csv'

##### On supprime l'index, on met une majuscule à probability (pour kaggle) et on supprime la colonne SeriousDlqin2yrs

In [12]:
from pandas import read_csv
final_submission=read_csv('./BatchPrediction/GMSCfulltrainm_Prediction2_Ensemble.csv', index_col=0)
final_submission.columns.values[1]='Probability'


In [14]:
del final_submission['SeriousDlqin2yrs']

##### On va enregistrer nos changements et envoyer nos résultats sur Kaggle

In [16]:
import kaggle
final_submission.to_csv('kagglepredGMSC.csv')
submission_file = 'kagglepredGMSC.csv'
kaggle.api.competition_submit(submission_file, "BigML ensemble", "GiveMeSomeCredit")

100%|██████████| 1.73M/1.73M [00:36<00:00, 50.5kB/s]


Successfully submitted to Give Me Some Credit

##### On peut récupérer notre score ainsi que les précédents, envoyé sur Kaggle, à l'aide de la commande suivante:

```python
kaggle.api.competitions_submissions_list("GiveMeSomeCredit")```